In [48]:
from moviepy.editor import VideoFileClip
import numpy as np
import librosa
import soundfile as sf
import noisereduce as nr
from scipy.io import wavfile
from scipy import signal
from scipy.signal import wiener
from scipy.ndimage import convolve1d


In [4]:
def video_to_audio(video_path, audio_path):
    video_clip = VideoFileClip(video_path)

    # Extract audio from the video
    audio_clip = video_clip.audio

    # Write the audio file
    audio_clip.write_audiofile(audio_path)

    # Close the video clip
    video_clip.close()

if __name__ == "__main__":
    video_file_path = "videos_audios\RIISE 2022 final video.mp4"  # Replace with your video file path
    audio_file_path = "videos_audios\output_audio.wav"  # Replace with your desired audio file path

    video_to_audio(video_file_path, audio_file_path)


MoviePy - Writing audio in videos_audios\output_audio.wav


MoviePy - Done.


In [89]:
def spectral_subtraction(audio_file, output_file, n_fft=2048, hop_length=512):
    # Load audio file
    audio, sr = librosa.load(audio_file, sr=None)

    # Estimate noise from the first few frames of audio
    noise_frames = audio[:n_fft]
    noise_mean = np.mean(np.abs(librosa.stft(noise_frames, n_fft=n_fft, hop_length=hop_length)), axis=1)

    # Compute STFT (Short-Time Fourier Transform) for audio
    stft_audio = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length)

    # Compute power spectrum for audio
    power_audio = np.abs(stft_audio)**2

    # Perform spectral subtraction
    power_clean = np.maximum(power_audio - noise_mean[:, np.newaxis], 0.0)

    # Reconstruct audio signal
    clean_audio = librosa.istft(np.sqrt(power_clean * np.exp(1j * np.angle(stft_audio))),
                                hop_length=hop_length)

    # Write cleaned audio to file
    sf.write(output_file, clean_audio, sr)

# Example usage
spectral_subtraction("videos_audios\output_final.wav", "videos_audios\output_cleaned.wav")

In [91]:
def remove_background_noise(audio_file, output_file, noise_level=50):
    # Step 1: Load Audio File
    y, sr = librosa.load(audio_file)

    # Step 2: Short-time Fourier Transform (STFT)
    D = librosa.stft(y)

    # Step 3: Spectral Subtraction
    magnitude = np.abs(D)
    power = magnitude ** 2
    noise_power = np.median(power, axis=1) * noise_level
    mask = np.where(power <= noise_power[:, np.newaxis], 0.0, 1.0)
    D_cleaned = mask * D

    # Step 4: Inverse Short-time Fourier Transform (ISTFT)
    y_cleaned = librosa.istft(D_cleaned)

    # Step 5: Save Cleaned Audio using scipy
    wavfile.write(output_file, sr, np.int16(y_cleaned*32767))  # Scaling to 16-bit integer

# Example usage
remove_background_noise("videos_audios\output_final.wav", "videos_audios\output_cleaned_final.wav")

In [72]:
def remove_beats_and_events(audio_file, output_file, threshold=1, beat_duration=0.6):
    # Step 1: Load Audio File
    y, sr = librosa.load(audio_file)

    # Step 2: Detect Beats or Loud Events
    # For simplicity, let's use amplitude thresholding
    onset_frames = librosa.onset.onset_detect(y=y, sr=sr, backtrack=True, hop_length=512)

    # Convert onset frames to time locations
    onset_times = librosa.frames_to_time(onset_frames, sr=sr)

    # Step 3: Attenuate or Remove Detected Sections
    for onset_time in onset_times:
        start_sample = librosa.time_to_samples(onset_time)
        end_sample = start_sample + int(sr * beat_duration)

        # Remove or attenuate the detected section (e.g., set amplitudes to zero)
        y[start_sample:end_sample] = 0  # Replace with your desired modification

    # Step 4: Save Cleaned Audio using scipy
    wavfile.write(output_file, sr, y)

# Example usage
remove_beats_and_events("videos_audios\output_cleaned2.wav", "videos_audios\output_cleaned3.wav")